In [73]:
%pip install psycopg2
%pip install nbformat

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: C:\Users\gogco\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: C:\Users\gogco\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [74]:
import nbformat
import psycopg2 as pg
import pandas as pd

### Введите данные для подключения к существующей или создание новой БД

In [75]:
DB_NAME = "stages"
DB_USER = "postgres"
DB_PASSWORD = ""
DB_HOST = "localhost"
DB_PORT = "5433"

In [76]:
# подключение к системной БД
conn = pg.connect(
    dbname="postgres",
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
conn.autocommit = True
cur = conn.cursor()

In [77]:
# попытка создания БД с указанным именем
cur.execute(f"SELECT 1 FROM pg_database WHERE datname = '{DB_NAME}';")
if not cur.fetchone():
    cur.execute(f"create database {DB_NAME} with owner = {DB_USER};")
    print(f"База данных {DB_NAME} успешно создана!")
else:
    print(f"База данных {DB_NAME} уже существует.")
cur.close()
conn.close()

База данных stages уже существует.


In [78]:
# подключение к существующей или созданной БД
conn = pg.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cur = conn.cursor()

### Создание таблиц и отношений БД SQL-запросом

In [79]:
try:
    with open("schema.sql", "r", encoding="utf-8") as f:
        sql_script = f.read()
    cur.execute(sql_script)
    conn.commit()

    # проверка, создались ли таблицы
    cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")
    tables = cur.fetchall()

    if tables:
        print("Таблицы успешно созданы!")
    else:
        print("Ошибка: таблицы не были созданы!")

except Exception as e:
    print(f"Ошибка при создании таблиц: {e}")
    conn.rollback()  # откат изменений, если была ошибка

Таблицы успешно созданы!


### > Выполнить preprocess_data.ipynb

In [80]:
# генерирует таблицы (файлы.csv) для дальнейшей заливки в БД
notebook_filename = "preprocess_data.ipynb"

with open(notebook_filename, "r", encoding="utf-8") as f:
    notebook = nbformat.read(f, as_version=4)

for cell in notebook.cells:
    if cell.cell_type == "code":
        exec(cell.source)

Дубликаты строк таблицы products_df = 19896
Дубликаты строк таблицы products_df после очистки = 0
Создан файл: product.csv
Создан файл: transaction.csv
Дубликаты строк таблицы address_df = 0
Дубликаты строк таблицы address_df после очистки = 0
Создан файл: address.csv
Дубликаты строк таблицы financial_situation_df = 0
Дубликаты строк таблицы financial_situation_df после очистки = 0
Создан файл: financial_situation.csv
Создан файл: customer.csv


In [81]:
# функция загрузки CSV в PostgreSQL
def load_csv_to_postgres(cur, csv_path, table_name):
    df = pd.read_csv(csv_path)
    # открытие файла и загрузка в PostgreSQL
    with open(csv_path, 'r', encoding='utf-8') as f:
        next(f)  # пропуск заголовков
        try:
            cur.copy_from(f, table_name, sep=',', null='')
            conn.commit()
            print(f"Данные из {csv_path} загружены в {table_name}")
        except Exception as e:
            conn.rollback()
            print(f"Ошибка при загрузке {csv_path} в {table_name}: {e}")

In [82]:
# загрузка данных в таблицы (порядок загрузки индивидуален в соответствии со схемой)
load_csv_to_postgres(cur, "data/customer.csv", "customer")
load_csv_to_postgres(cur, "data/financial_situation.csv", "financial_situation")
load_csv_to_postgres(cur, "data/address.csv", "address")
load_csv_to_postgres(cur, "data/product.csv", "product")
load_csv_to_postgres(cur, "data/transaction.csv", "transaction")

cur.close()
conn.close()

Данные из data/customer.csv загружены в customer
Данные из data/financial_situation.csv загружены в financial_situation
Данные из data/address.csv загружены в address
Данные из data/product.csv загружены в product
Данные из data/transaction.csv загружены в transaction
